In [ ]:
#Preliminary predictive model for clothing predictions

In [1]:
import numpy as np
import pandas as pd
import random
import datetime

In [2]:
#Creating the dataset

#features

"""
Weather features:
1. Temperature (Celcius) (-20 to 25)
2. Humidity (%) (relative) (0 to 100)
3. Probability of Precipitation (%) (0 to 100)
4. Wind speed (mph)

User features:
1. Age (7 to 70)
2. Weight (kg)
3. Sex
4. Height (cm)
5. Fat percentage
"""

#output
"""
output will be in the form of a boolean array with each index presenting different clothing articles
0. short sleve shirt + long pants (always on)
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

'\noutput will be in the form of a boolean array with each index presenting different clothing articles\n0. short sleve shirt + long pants (always on)\n1. Thermal (both top and bottom)\n2. Jacket/Hoodie\n3. light downjacket\n4. Thick down/Winter down\n5. Wind breaker\n6. Umbrella/Raincoat\n'

In [3]:
from datetime import date
data = []
currentdate = date.today()
for i in range(100000):
    date = (currentdate - datetime.timedelta(i)).strftime("%d/%m/%Y") #display in day/month/year
    temperature = random.randint(0, 45) - 20
    humidity = random.randint(0, 100)
    precipitation = random.randint(0, 100)
    windspeed = random.randint(0, 40) #mph
    age = random.randint(7, 70)
    weight = random.randint(20, 120) #kg
    height = random.randint(110, 200) #cm
    sex = random.randint(0,1) # 0:M 1:F
    #fat percentage formula from
    #https://www.gaiam.com/blogs/discover/how-to-calculate-your-ideal-body-fat-percentage
    
    BMI = weight/(height**2)*10000
    if sex == 0:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 16.2
        
    elif sex == 1:
        fatpercentage = (1.20 * BMI) + (0.23 * age) - 5.4
        
    datapoint = (date, temperature, humidity, precipitation, windspeed, age, weight, height, sex, fatpercentage, BMI)
    data.append(datapoint)
    

In [4]:
columnstring = 'date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi'
column = columnstring.split(',')
df = pd.DataFrame(data, columns = column)

In [5]:
df[:100]

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi
0,11/01/2022,-5,87,35,40,70,97,138,0,61.021613,50.934678
1,10/01/2022,23,34,52,26,64,71,143,1,50.984629,34.720524
2,09/01/2022,7,96,67,11,54,92,171,0,33.975207,31.462672
3,08/01/2022,15,87,74,21,26,71,141,1,43.434987,35.712489
4,07/01/2022,-9,91,48,9,68,86,157,1,52.107824,34.889854
...,...,...,...,...,...,...,...,...,...,...,...
95,08/10/2021,17,41,4,36,10,89,194,1,25.277086,23.647571
96,07/10/2021,-18,73,85,25,14,57,176,0,9.101612,18.401343
97,06/10/2021,11,18,21,28,70,22,130,1,26.321302,13.017751
98,05/10/2021,-15,99,27,21,36,22,160,0,2.392500,8.593750


In [10]:
#layers

"""
1. Thermal (both top and bottom)
2. Jacket/Hoodie
3. light downjacket
4. Thick down/Winter down
5. Wind breaker
6. Umbrella/Raincoat
"""

#rules
"""
1. If temp < 5 : Thermal = True and thickdown = True
2. If temp < 15 : Jacket = True
3. If temp < 10 : lightdown = True
4. If windspeed > 25 : windbreaker = True
5. If precipitation > 50%: umbrella/raincoast = True


cold resistance categories:
low: 0 - 24
mid-low: 25 - 49
mid-high: 50 - 74
high: 75 - 99
"""


#cold resistance calculation

male_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}


female_weights = {
    'age' : 40,
    'bmi' : 30,
    'fatpercentage' : 30
}

# cold resistance categories:
# low: 0 - 24
# mid-low: 25 - 49
# mid-high: 50 - 74
# high: 75 - 99
cold_resistance_scores = {
    'low' : 25,
    'mid-low' : 50,
    'mid-high' : 75,
    'high' : 100
}




In [11]:
#returns cold resistance score
#formula:

#cold_resistance = age_score * gender_age_weight + bmi_score + gender_bmi_weight + gender_fatpercentage_weight
def coldresistance(sex, age, fatpercentage, BMI):
    #female case
    if sex == 1:
        if (age <= 12 or age >= 55):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 13 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 14):
            fatpercentage_score = cold_resistance_scores['low']
        elif (14 < fatpercentage <= 20):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (20 < fatpercentage <= 25): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
      
    #men
    else:
        if (age <= 12 or age >= 55):
            #low immunity to cold at younger or older ages
            age_score = cold_resistance_scores['low']
            #mid low immunity between 13 and 25 years old
        elif 13 <= age <= 25:
            age_score = cold_resistance_scores['mid-low']
            #highest immunity between 26 and 40 years old
        elif 26 <= age <= 40:
            age_score = cold_resistance_scores['high']
        else:
            #mid high immunity between 41 and 54 years old
            age_score = cold_resistance_scores['mid-high']

        #fatpercentage ranges referenced from https://www.medicalnewstoday.com/articles/body-fat-percentage-chart#chart

        if (fatpercentage <= 8):
            fatpercentage_score = cold_resistance_scores['low']
        elif (8 < fatpercentage <= 15):
            fatpercentage_score = cold_resistance_scores['mid-low']

        elif (15 < fatpercentage <= 24): 
            fatpercentage_score = cold_resistance_scores['mid-high']

        else:
            fatpercentage_score = cold_resistance_scores['high']

        #severly underweight
        if (BMI <= 15):
            bmi_score = cold_resistance_scores['low']
        #underweight
        elif (15 < BMI <= 18):
            bmi_score = cold_resistance_scores['mid-low']
        #healthy
        elif (18 < BMI <= 25):
            bmi_score = cold_resistance_scores['mid-high']
        #overweight
        else:
            bmi_score = cold_resistance_scores['high']
            
    #female case
    if sex == 1:
        cold_resistance = age_score/100*female_weights['age'] + bmi_score/100*female_weights['bmi'] + fatpercentage_score/100*female_weights['fatpercentage']
    
    #male case
    elif sex == 0:
        cold_resistance = age_score/100*male_weights['age'] + bmi_score/100*male_weights['bmi'] + fatpercentage_score/100*male_weights['fatpercentage']
    
    return cold_resistance

In [17]:
df['cold_resistance'] = df.apply(lambda row : coldresistance(row['sex'], row['age'], row['fatpercentage'], row['bmi']), axis = 1)

In [19]:
df[:100]

,date,temperature,humidity,precipitation,windspeed,age,weight,height,sex,fatpercentage,bmi,cold_resistance
0,11/01/2022,-5,87,35,40,70,97,138,0,61.021613,50.934678,70.0
1,10/01/2022,23,34,52,26,64,71,143,1,50.984629,34.720524,70.0
2,09/01/2022,7,96,67,11,54,92,171,0,33.975207,31.462672,90.0
3,08/01/2022,15,87,74,21,26,71,141,1,43.434987,35.712489,100.0
4,07/01/2022,-9,91,48,9,68,86,157,1,52.107824,34.889854,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,08/10/2021,17,41,4,36,10,89,194,1,25.277086,23.647571,62.5
96,07/10/2021,-18,73,85,25,14,57,176,0,9.101612,18.401343,57.5
97,06/10/2021,11,18,21,28,70,22,130,1,26.321302,13.017751,47.5
98,05/10/2021,-15,99,27,21,36,22,160,0,2.392500,8.593750,55.0
